### Install dependencies

In [1]:
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent wget torf && apt install python3-libtorrent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.3 MB/s  0:00:00
  DEPRECATION: Building 'wget' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enfo

### Download torrent

In [9]:
from google.colab import files

uploaded = files.upload()  # Opens file chooser
torrent_file_path = list(uploaded.keys())[0]
print(f"Uploaded: {torrent_file_path}")


Saving Jurassic World Rebirth (2025) [720p] [WEBRip] [YTS.MX].torrent to Jurassic World Rebirth (2025) [720p] [WEBRip] [YTS.MX].torrent
Uploaded: Jurassic World Rebirth (2025) [720p] [WEBRip] [YTS.MX].torrent


In [10]:
import zipfile
import os
import libtorrent as lt
import time
import datetime
import shutil

params = {
    'save_path': '/content/Torrent/',
    'storage_mode': lt.storage_mode_t(2),
}

ses = lt.session()
ses.listen_on(6881, 6891)
# link = input("Input Torrent file link or Magnet link and Press Enter: ")

# if link.endswith('.torrent'):
#     import wget
#     from torf import Torrent

#     if os.path.exists('torrent.torrent'):
#         os.remove('torrent.torrent')

#     wget.download(link, 'torrent.torrent')
#     t = Torrent.read('torrent.torrent')
#     link = str(t.magnet(name=True, size=False, trackers=False, tracker=False))



# print(link)
# handle = lt.add_magnet_uri(ses, link, params)

if torrent_file_path.endswith('.torrent'):
    info = lt.torrent_info(torrent_file_path)
    handle = ses.add_torrent({'ti': info, 'save_path': params['save_path']})


# change the 0 to a 1 to download sequentially
handle.set_sequential_download(0)
ses.start_dht()
begin = time.time()

print(datetime.datetime.now())
print('Downloading Metadata...')

while not handle.has_metadata():
    time.sleep(1)

print('Got Metadata, Starting Torrent Download...')
print("Starting", handle.name())

while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']
    print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
           s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")
print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
print(datetime.datetime.now())

KeyboardInterrupt: Interrupted by user

### Zip the files

In [ ]:
print('Zipping started...')

ziph = zipfile.ZipFile('Downloaded-Torrent.zip', 'w', zipfile.ZIP_DEFLATED)
path = 'content/Torrent/'
for root, dirs, files in os.walk(path):
    for file in files:
        ziph.write(os.path.join(root, file),
                   os.path.relpath(os.path.join(root, file),
                                   os.path.join(path, '..')))
ziph.close()

print('\nALL DONE!')